In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import os
import numpy as np
import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.layers import Masking

Using TensorFlow backend.
C:\Users\silva\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\silva\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\silva\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\silva\AppData\Roa

In [2]:
days = 30

In [3]:
def scale(training_set):
    # Feature Scaling
    sc = MinMaxScaler(feature_range = (0, 1))
    training_set_scaled = sc.fit_transform(training_set)
    # Creating a data structure with 'days' time-steps and 1 output
    X_train = []
    y_train = []
    for i in range(days, training_set.shape[0]):
        X_train.append(training_set_scaled[i-days:i, 0])
        y_train.append(training_set_scaled[i, 0])
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    return X_train, y_train, sc

In [4]:
def train_LSTM(X_train, y_train):
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape = (X_train.shape[1], 1)))
    #Adding the first LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    model.add(Dropout(0.2))
    # Adding a second LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))
    # Adding a third LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))
    # Adding a fourth LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50))
    model.add(Dropout(0.2))
    # Adding the output layer
    model.add(Dense(units = 1))

    # Compiling the RNN
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')

    # Fitting the RNN to the Training set
    model.fit(X_train, y_train, epochs = 30, batch_size = 32)
    return model

In [5]:
def predict(dataset_train, dataset_test, model, sc):
    dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)
    inputs = dataset_total[len(dataset_total) - dataset_test.shape[0] - days:].values
    inputs = inputs.reshape(-1,1)
    inputs = sc.transform(inputs)
    X_test = []
    for i in range(days, inputs.shape[0]):
        X_test.append(inputs[i-days:i, 0])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    print(X_test.shape)
    # (273, 'days', 1)
    predicted_stock_price = model.predict(X_test)
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)
    return X_test, predicted_stock_price

In [6]:
def plot_results(dataset_test, predicted_stock_price):
    # Visualising the results
    plt.plot(dataset_test.values, color = 'red', label = 'Real')
    plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted')
    plt.xticks(np.arange(0,dataset_test.shape[0],50))
    plt.xlabel('Time')
    plt.ylabel('Index')
    plt.legend()
    plt.show()

In [7]:
def impute_with_prediction(df_orig, df_imputed):
    for i in df_orig.index:
        if pd.isna(df_orig.loc[i]):
            df_orig.at[i] = df_imputed.loc[i]
    return df_orig

In [8]:
INPUT = 'output/\CompleteWeeklyIndexes'
df = pd.read_csv(INPUT+'.csv', parse_dates=True, index_col=0)

In [ ]:
for i in range(0,30):
    df_train_complete = None
    df_test_complete = None
    for col in df.columns:
        test_set = df.loc['2015-01-02':, col].fillna(-1).values #Selecting test data
        training_set = df.loc['2000-01-01':'2015-01-01', col].fillna(-1).values #Selecting training data
        training_set = training_set.reshape(-1, 1)
        X_train, y_train, sc = scale(training_set)
        model = train_LSTM(X_train, y_train)
        # Dados de treinamento
        predicted_stock_price_train = model.predict(X_train)
        predicted_stock_price_train = sc.inverse_transform(predicted_stock_price_train)
        #plot_results(df.loc['2000-01-01':'2015-01-01', col].fillna(-1), predicted_stock_price_train)
        # Dados de teste
        X_test, predicted_stock_price = predict(df.loc['2000-01-01':'2015-01-01', col].fillna(-1), df.loc['2015-01-02':, col].fillna(-1), model, sc)
        #plot_results(df.loc['2015-01-02':, col].fillna(-1), predicted_stock_price)

        vazio = np.empty(days)
        predicted_stock_price_train = np.append(vazio, predicted_stock_price_train)
        series_complete_train = impute_with_prediction(df.loc['2000-01-01':'2015-01-01', col], pd.DataFrame(index=df.loc['2000-01-01':'2015-01-01', col].index, data=predicted_stock_price_train))
        df_train_complete = pd.concat([df_train_complete, pd.DataFrame(index=df.loc['2000-01-01':'2015-01-01', col].index, data=series_complete_train, columns=[col])], axis=1, join='outer')
        df_test_complete = pd.concat([df_test_complete, pd.DataFrame(data=predicted_stock_price, index = df.loc['2015-01-02':, col].index, columns = [col])], axis=1, join='outer')
    df_train_complete.to_csv(r'output\/lstm\/CompleteIndexesWeeklyTrainLSTM30_n'+str(i)+'.csv', index = True)
    df_test_complete.to_csv(r'output\/lstm\/CompleteIndexesWeeklyTestLSTM30_n'+str(i)+'.csv', index = True)

Epoch 1/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0893
Epoch 2/30
752/752 [==============================] - 3s 4ms/step - loss: 0.0257
Epoch 3/30
752/752 [==============================] - 3s 5ms/step - loss: 0.0215
Epoch 4/30
752/752 [==============================] - 3s 5ms/step - loss: 0.0186
Epoch 5/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0196
Epoch 6/30
752/752 [==============================] - 3s 5ms/step - loss: 0.0180
Epoch 7/30
752/752 [==============================] - 3s 5ms/step - loss: 0.0186
Epoch 8/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0179
Epoch 9/30
752/752 [==============================] - 3s 5ms/step - loss: 0.0167
Epoch 10/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0170
Epoch 11/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0174
Epoch 12/30
752/752 [==============================] - 3s 5ms/step - loss: 0.0170
Epoch 13/30
752/752 [===

752/752 [==============================] - 3s 5ms/step - loss: 0.0404
Epoch 11/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0378
Epoch 12/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0385
Epoch 13/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0379
Epoch 14/30
752/752 [==============================] - 3s 5ms/step - loss: 0.0383
Epoch 15/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0428
Epoch 16/30
752/752 [==============================] - 3s 5ms/step - loss: 0.0402
Epoch 17/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0373
Epoch 18/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0387
Epoch 19/30
752/752 [==============================] - 3s 5ms/step - loss: 0.0369
Epoch 20/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0383
Epoch 21/30
752/752 [==============================] - 3s 5ms/step - loss: 0.0413
Epoch 22/30
752/752 [=======

752/752 [==============================] - 4s 5ms/step - loss: 0.0099
Epoch 20/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0117
Epoch 21/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0089
Epoch 22/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0091
Epoch 23/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0091
Epoch 24/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0095
Epoch 25/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0086
Epoch 26/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0093
Epoch 27/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0098
Epoch 28/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0087
Epoch 29/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0090
Epoch 30/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0092
(272, 30, 1)
Epoch 1/30
752/

752/752 [==============================] - 4s 5ms/step - loss: 0.0102
Epoch 29/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0098
Epoch 30/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0098
(272, 30, 1)
Epoch 1/30
752/752 [==============================] - 10s 13ms/step - loss: 0.1333
Epoch 2/30
752/752 [==============================] - 2s 3ms/step - loss: 0.0435
Epoch 3/30
752/752 [==============================] - 2s 3ms/step - loss: 0.0394
Epoch 4/30
752/752 [==============================] - 2s 3ms/step - loss: 0.0396
Epoch 5/30
752/752 [==============================] - 2s 3ms/step - loss: 0.0352
Epoch 6/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0357
Epoch 7/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0337
Epoch 8/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0339
Epoch 9/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0345
Epoch 10/30
752/752 [=

752/752 [==============================] - 3s 4ms/step - loss: 0.0099
Epoch 8/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0108
Epoch 9/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0099
Epoch 10/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0099
Epoch 11/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0098
Epoch 12/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0098
Epoch 13/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0102A: 0s - loss: 
Epoch 14/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0100
Epoch 15/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0096
Epoch 16/30
752/752 [==============================] - 4s 6ms/step - loss: 0.0100
Epoch 17/30
752/752 [==============================] - 4s 6ms/step - loss: 0.0102
Epoch 18/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0098
Epoch 19/30
752/

...

752/752 [==============================] - 6s 8ms/step - loss: 0.0028
Epoch 29/30
752/752 [==============================] - 6s 9ms/step - loss: 0.0028
Epoch 30/30
752/752 [==============================] - 6s 8ms/step - loss: 0.0028
(272, 30, 1)
Epoch 1/30
752/752 [==============================] - 86s 114ms/step - loss: 0.0063
Epoch 2/30
752/752 [==============================] - 8s 10ms/step - loss: 0.0063
Epoch 3/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0063
Epoch 4/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0062
Epoch 5/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0061
Epoch 6/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0062
Epoch 7/30
752/752 [==============================] - 6s 8ms/step - loss: 0.0062
Epoch 8/30
752/752 [==============================] - 7s 10ms/step - loss: 0.0061
Epoch 9/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0062
Epoch 10/30
752/752

752/752 [==============================] - 7s 10ms/step - loss: 0.0294
Epoch 8/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0294
Epoch 9/30
752/752 [==============================] - 7s 10ms/step - loss: 0.0293
Epoch 10/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0289
Epoch 11/30
752/752 [==============================] - 7s 10ms/step - loss: 0.0290
Epoch 12/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0295
Epoch 13/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0290
Epoch 14/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0291
Epoch 15/30
752/752 [==============================] - 7s 10ms/step - loss: 0.0290
Epoch 16/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0289
Epoch 17/30
752/752 [==============================] - 7s 10ms/step - loss: 0.0293
Epoch 18/30
752/752 [==============================] - 6s 8ms/step - loss: 0.0292
Epoch 19/30
752/752 [====

752/752 [==============================] - 8s 10ms/step - loss: 0.0041
Epoch 17/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0040
Epoch 18/30
752/752 [==============================] - 8s 10ms/step - loss: 0.0039
Epoch 19/30
752/752 [==============================] - 8s 10ms/step - loss: 0.0038
Epoch 20/30
752/752 [==============================] - 7s 10ms/step - loss: 0.0040
Epoch 21/30
752/752 [==============================] - 8s 10ms/step - loss: 0.0036
Epoch 22/30
752/752 [==============================] - 8s 10ms/step - loss: 0.0035
Epoch 23/30
752/752 [==============================] - 7s 10ms/step - loss: 0.0036
Epoch 24/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0037
Epoch 25/30
752/752 [==============================] - 7s 10ms/step - loss: 0.0035
Epoch 26/30
752/752 [==============================] - 8s 10ms/step - loss: 0.0035
Epoch 27/30
752/752 [==============================] - 7s 10ms/step - loss: 0.0035
Epoch 28/30
752/75

752/752 [==============================] - 8s 10ms/step - loss: 0.0159
Epoch 25/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0166
Epoch 26/30
752/752 [==============================] - 7s 10ms/step - loss: 0.0163
Epoch 27/30
752/752 [==============================] - 8s 10ms/step - loss: 0.0161
Epoch 28/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0160
Epoch 29/30
752/752 [==============================] - 8s 10ms/step - loss: 0.0174
Epoch 30/30
752/752 [==============================] - 7s 10ms/step - loss: 0.0154
(272, 30, 1)
Epoch 1/30
752/752 [==============================] - 303s 402ms/step - loss: 0.0496
Epoch 2/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0101
Epoch 3/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0059
Epoch 4/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0052
Epoch 5/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0044
Epoch 6

752/752 [==============================] - 11s 15ms/step - loss: 0.0109
Epoch 3/30
752/752 [==============================] - 11s 15ms/step - loss: 0.0092
Epoch 4/30
752/752 [==============================] - 9s 13ms/step - loss: 0.0083
Epoch 5/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0086
Epoch 6/30
752/752 [==============================] - 9s 13ms/step - loss: 0.0079
Epoch 7/30
752/752 [==============================] - 10s 13ms/step - loss: 0.0081
Epoch 8/30
752/752 [==============================] - 10s 13ms/step - loss: 0.0078
Epoch 9/30
752/752 [==============================] - 10s 13ms/step - loss: 0.0074
Epoch 10/30
752/752 [==============================] - 10s 13ms/step - loss: 0.0071
Epoch 11/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0073
Epoch 12/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0086
Epoch 13/30
752/752 [==============================] - 9s 13ms/step - loss: 0.0075
Epoch 14/30
752/7